<a href="https://colab.research.google.com/github/Pheonix10101/PRCV_p_5/blob/main/Task_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
"""
Project 5: Recognition using Deep Networks

Author: Samruddhi Raut

This file contain following tasks

Task 4:

Design your own experiment

"""

#Import statement
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.backends.cudnn as cudnn
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision import datasets
from matplotlib import pyplot as plt
import time
import CNN

In [ ]:
# set the random seed
torch.manual_seed(42)
cudnn.enabled = False

BATCH_SIZE_TRAIN = [64,128,192]
BATCH_SIZE_TEST = 1000
EPOCHS = [5, 7, 9]
LEARNING_RATE = 0.05
MOMENTUM = 0.5
CONV_FILTER_SIZE = [5,10,15]
DROPOUT_RATE = [0.5]
LOG_INTERVAL = 10

**Defining a variation class**

In [ ]:
class Variation(nn.Module):
    def __init__(self, conv_filter_size, dropout_rate):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 10, (conv_filter_size, conv_filter_size))
        self.conv2 = nn.Conv2d(10, 20, (conv_filter_size, conv_filter_size))
        self.conv2_drop = nn.Dropout2d(dropout_rate)
        self.flat1 = nn.Flatten()
        outer = conv_filter_size // 2
        size = ((28 - 2 * outer) // 2 - 2 * outer) // 2
        self.fc1 = nn.Linear(20 * size * size, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = self.conv1(x)# A convolution layer with 10 5x5 filters
        x = F.relu(F.max_pool2d(x, (2, 2)))# A max pooling layer with a 2x2 window and a ReLU function applied
        x = self.conv2(x)# A convolution layer with 20 5x5 filters
        x = self.conv2_drop(x)# A dropout layer with a 0.5 dropout rate (50%)
        x = F.relu(F.max_pool2d(x, (2, 2))) # A max pooling layer with a 2x2 window and a ReLU function applied
        x = F.relu(self.fc1(self.flat1(x))) # A flattening operation followed by a fully connected Linear layer with 50 nodes and a ReLU function on the output
        x = F.log_softmax(self.fc2(x), dim = 1) # A final fully connected Linear layer with 10 nodes and the log_softmax function applied to the output

        return x


In [ ]:
def get_loader(batch_size_train):
    transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))])
    train_mnist = datasets.MNIST('mnist',
                                 train = True,
                                 download = True,
                                 transform = transform
                                 )
    test_mnist = datasets.MNIST('mnist',
                                train = False,
                                download = True,
                                transform = transform
                                )
    train_loader = DataLoader(dataset = train_mnist,
                              batch_size = batch_size_train,
                              shuffle = True,
                              num_workers = 4)
    test_loader = DataLoader(dataset = test_mnist,
                             batch_size = BATCH_SIZE_TEST,
                             shuffle = True,
                             num_workers = 4)

    return train_loader, test_loader


**defening train_and_test**

train the model on training set epochs# times
    :param train_loader:    dataloader of training set
    :param test_loader:     dataloader of testing set
    :param batch_size:      batch_size of training
    :param epochs:          times complete pass through the training data
    :param conv_filter_size:size of conv filter
    :param dropout_rate:    dropout rate
    :param learning_rate:   learning rate
    :param momentum:        refers to inertia
    :param log_interval:    interval between every two recordings of loss
    

In [ ]:
def train_and_test(train_loader, test_loader, batch_size, epochs, conv_filter_size, dropout_rate,
                   learning_rate, momentum, log_interval):
    
    # initialize model and optimizer
    network = Variation(conv_filter_size, dropout_rate)
    optimizer = optim.SGD(network.parameters(), lr = learning_rate, momentum = momentum)
    train_losses = []
    train_counter = []
    test_losses_training_set = []
    test_losses = []
    test_counter = [i * len(train_loader.dataset) for i in range(epochs + 1)]

    CNN.test_network(network, train_loader, test_losses_training_set, 0, False)
    CNN.test_network(network, test_loader, test_losses, 0, True)
    for epoch in range(1, epochs + 1):
        CNN.train_network(network, train_loader, epoch, optimizer, log_interval, train_losses,
                                  train_counter, False)
        CNN.test_network(network, train_loader, test_losses_training_set, epoch, False)
        CNN.test_network(network, test_loader, test_losses, epoch, True)

    
    fig = plt.figure()
    plt.plot(train_counter, train_losses, color = 'green')
    plt.scatter(test_counter, test_losses, color = 'red', alpha = 1)
    plt.legend(['Train Loss', 'Test Loss on training set'], loc = 'upper right')
    plt.title('Batch size: {} Epoch #: {} Conv filter size: {} Dropout rate: {}'.format(batch_size,
                                                                                        epochs,
                                                                                        conv_filter_size,
                                                                                        dropout_rate))
    plt.xlabel('number of training examples seen')
    plt.ylabel('negative log likelihood loss')
   
    fig.show()


Defining a main function

In [ ]:
def main():
    c = 1
    for i in BATCH_SIZE_TRAIN:
        train_loader, test_loader = get_loader(i)
        for j in EPOCHS:
            for p in CONV_FILTER_SIZE:
                for q in DROPOUT_RATE:
                    start = time.time()
                    print('Training and evaluating for variation %d:' % c)
                    print('\tBatch size: {}'.format(i))
                    print('\tEpoch #: {}'.format(j))
                    print('\tConv filter size: {}'.format(p))
                    print('\tDropout rate: {}\n'.format(q))
                    train_and_test(train_loader, test_loader, i, j, p, q, LEARNING_RATE, MOMENTUM, LOG_INTERVAL)
                    end = time.time()
                    print("Time cost: %.2fs" % (end - start))
                    c += 1


if __name__ == "__main__":
    main()